In [1]:
import time
import sys
sys.path.append('./mrnet/src')

import mrnet.network
import mrnet.core.mol_entry
import mrnet.core.reactions
import mrnet.core.rates
sys.modules['pymatgen.reaction_network'] = mrnet.network
sys.modules['pymatgen.entries.mol_entry'] = mrnet.core.mol_entry
sys.modules['pymatgen.reaction_network.reaction'] =  mrnet.core.reactions
sys.modules['pymatgen.reaction_network.reaction_rates'] = mrnet.core.rates

In [2]:
import pickle
# currently, these will only load using commit 222ad71 of mrnet
ledc_no_hop_file = "./-2.15V_ledc_lemc_network_pickle_wo_hop_1123"


with open(ledc_no_hop_file, 'rb') as f:
    ledc_no_hop_reaction_network = pickle.load(f)

In [3]:
from RNMC import *

In [4]:
ledc_rns = ReactionNetworkSerialization(ledc_no_hop_reaction_network)

In [5]:
ledc_initial_state = np.zeros(ledc_rns.number_of_species)
Li_plus_index = ledc_rns.find_index_from_mol_graph('./Li.xyz', 1)
EC_index = ledc_rns.find_index_from_mol_graph('./mrnet/test_files/reaction_network_files/EC.xyz',0)
LEDC_index = ledc_rns.find_index_from_mol_graph('./mrnet/test_files/reaction_network_files/LEDC.xyz',0)
ledc_initial_state[Li_plus_index] = 30
ledc_initial_state[EC_index] = 30

In [6]:
# this should produce the sequence 143, 121, 20, 13, 2, 1
plasma_file = '/tmp/plasma_RNMC'
plasma_process = start_plasma_server(1,plasma_file)
ledc_mcb = MonteCarloBundler(8,ledc_rns,ledc_initial_state,5,range(1000),plasma_file)
ledc_mcb.pp_pathways(LEDC_index)
plasma_process.terminate()

143 occurrences:
EC + Li_plus -> LiEC_plus_bi                      -0.5731184586145162
LiEC_plus_bi -> LiEC_bi                           -0.01966738607516172
LiEC_bi -> LiEC_RO1                               -1.2074918081634678
LiEC_RO1 -> LiEC_minus_RO1                        -0.7467395352065069
EC + Li_plus -> LiEC_plus_bi                      -0.5731184586145162
LiEC_minus_RO1 -> C2H4 + LiCO3_minus              -1.6367992134601081
LiCO3_minus + LiEC_plus_bi -> LEDC                -1.5200711618599598

121 occurrences:
EC + Li_plus -> LiEC_plus_bi                      -0.5731184586145162
LiEC_plus_bi -> LiEC_bi                           -0.01966738607516172
LiEC_bi -> LiEC_RO2                               -1.0850033295464527
LiEC_RO2 -> LiEC_minus_RO2                        -0.7617288706602268
EC + Li_plus -> LiEC_plus_bi                      -0.5731184586145162
LiEC_minus_RO2 -> LiCO3_minus + C2H4              -1.7442983566234034
LiCO3_minus + LiEC_plus_bi -> LEDC                -1.